# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from vacationAPI import googleKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
vacationCity = pd.read_csv('../WeatherPy/cleanCity.csv')
del vacationCity['City_ID']
vacationCity

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Atuona,85,PF,1603738920,73,-9.80,-139.03,79.14,19.33
1,Makakilo City,1,US,1603738907,83,21.35,-158.09,75.20,3.36
2,Port Lincoln,2,AU,1603739046,88,-34.73,135.87,53.98,8.32
3,Barra do Corda,100,BR,1603739046,42,-5.50,-45.25,94.98,2.89
4,Cape Town,0,ZA,1603738910,77,-33.93,18.42,68.00,11.41
...,...,...,...,...,...,...,...,...,...
563,Jumla,0,NP,1603738870,59,29.27,82.18,42.28,4.63
564,Gamba,100,GA,1603738922,86,-2.65,10.00,76.71,7.31
565,Diapaga,0,BF,1603739120,23,12.07,1.79,84.33,5.68
566,Sitka,90,US,1603738914,87,57.05,-135.33,51.80,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
gmaps.configure(api_key = googleKey)

In [30]:
locations = vacationCity[["Latitude", "Longitude"]]
weights = vacationCity["Humidity"].astype(float)
heatMap = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius=1)
heatMap.add_layer(heat_layer)
heatMap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
vacationCity = vacationCity.dropna()
vacationCity.dtypes
fitVacation = vacationCity.loc[(vacationCity['Max Temp'] >= 70) & (vacationCity['Max Temp'] <= 80) & (vacationCity['Humidity'] <= 85) & (vacationCity['Humidity'] >= 15)& (vacationCity['Wind Speed'] <= 15) & (vacationCity['Cloudiness'] <= 25)]
fitVacation.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
1,Makakilo City,1,US,1603738907,83,21.35,-158.09,75.20,3.36
9,Chake Chake,0,TZ,1603739047,81,-5.25,39.77,76.46,6.15
39,Talara,0,PE,1603739051,57,-4.58,-81.27,78.80,9.17
53,Arraial do Cabo,20,BR,1603738889,65,-22.97,-42.03,78.80,8.05
58,Isangel,16,VU,1603738885,81,-19.55,169.27,73.69,8.90


In [32]:
fitVacation['Hotel Name'] = " "
fitVacation = fitVacation.reset_index(drop=True)

<ipython-input-32-67b6fc08f594>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fitVacation['Hotel Name'] = " "


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
params = {'key': googleKey,
          'type': 'lodging',
          'radius': 5000}

def findnearby(measures):
    baseURL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(baseURL, params = params)
    data = response.json()
    return data
    
for index, row in fitVacation.iterrows():
    city = row["City"]
    print(f'Hotel Search Number: {index} | {city}')
    lat = row["Latitude"]
    long = row["Longitude"]
    params["location"] = f"{lat},{long}"
    hotelData = findnearby(params)
    
    try:
        fitVacation.loc[index,'Hotel Name'] = hotelData['results'][0]['name']

    except:
        print(f"No values for {index} | {city}")

Hotel Search Number: 0 | Makakilo City
Hotel Search Number: 1 | Chake Chake
Hotel Search Number: 2 | Talara
Hotel Search Number: 3 | Arraial do Cabo
Hotel Search Number: 4 | Isangel
Hotel Search Number: 5 | Assiut
Hotel Search Number: 6 | Sur
Hotel Search Number: 7 | Qandala
No values for 7 | Qandala
Hotel Search Number: 8 | Lingao
Hotel Search Number: 9 | Awjilah
No values for 9 | Awjilah
Hotel Search Number: 10 | Mitsamiouli
Hotel Search Number: 11 | ‘Ewa Beach
Hotel Search Number: 12 | Tadine
No values for 12 | Tadine
Hotel Search Number: 13 | Pisco
Hotel Search Number: 14 | Riyadh
Hotel Search Number: 15 | Lompoc
Hotel Search Number: 16 | Jalu
Hotel Search Number: 17 | Manokwari
Hotel Search Number: 18 | Ouadda
No values for 18 | Ouadda
Hotel Search Number: 19 | Livingstone
Hotel Search Number: 20 | Khorramshahr
Hotel Search Number: 21 | Rivadavia
Hotel Search Number: 22 | Mercedes
Hotel Search Number: 23 | Cap Malheureux
Hotel Search Number: 24 | Salalah
Hotel Search Number: 25 | 

In [34]:
fitVacation

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Makakilo City,1,US,1603738907,83,21.35,-158.09,75.20,3.36,Marriott's Ko Olina Beach Club
1,Chake Chake,0,TZ,1603739047,81,-5.25,39.77,76.46,6.15,Hotel Archipelago
2,Talara,0,PE,1603739051,57,-4.58,-81.27,78.80,9.17,Hostal Oro Negro
3,Arraial do Cabo,20,BR,1603738889,65,-22.97,-42.03,78.80,8.05,Pousada Porto Praia
4,Isangel,16,VU,1603738885,81,-19.55,169.27,73.69,8.90,Volcano Sea View Tree House
5,Assiut,0,EG,1603739054,53,27.18,31.18,75.20,11.41,Florence Hotel Assiut
6,Sur,0,OM,1603739056,61,22.57,59.53,72.07,1.79,Sur Plaza Hotel
7,Qandala,21,SO,1603739057,40,11.47,49.87,77.88,4.56,
8,Lingao,5,CN,1603739061,84,19.91,109.69,73.17,3.94,Lingao Ditai Inn
9,Awjilah,0,LY,1603738917,51,29.11,21.29,71.91,6.73,


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fitVacation.iterrows()]
locations = fitVacation[["Latitude", "Longitude"]]

In [36]:
#markers = gmaps.marker_layer(locations, info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])

locations = fitVacation[["Latitude", "Longitude"]]
weights = fitVacation["Humidity"].astype(float)
hotelMap = gmaps.figure()
heatLayer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,point_radius=1)
hotelMap.add_layer(heatLayer)
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
hotelMap.add_layer(markers)
hotelMap

Figure(layout=FigureLayout(height='420px'))